In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ibge_faixa_etaria.csv to ibge_faixa_etaria.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **CARREGANDO A PRIMEIRA BASE DE DADOS PARA PROJETAR 2022**

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np

# Leia o conjunto de dados original
data = pd.read_csv('/content/drive/MyDrive/Dados para Analise/IBGE/ibge_faixa_etaria.csv', delimiter=';')
data['ano'] = pd.to_datetime(data['ano'], format='%Y', errors='coerce') + pd.DateOffset(days=1, months=1)
data['ano'] = data['ano'].dt.strftime('%Y-%m')  # Transforme a coluna 'ano' em string no formato YYYY-MM
data['ano'] = pd.to_datetime(data['ano'])  # Transforme de volta para o formato de data
data['ano'].sort_index(inplace=True)

# Defina o ano para o qual você deseja fazer projeções
year_to_project = 2022

def project_cod_ibge_sex(data, columns_to_project, year):
    unique_cod_ibge = data['cod_ibge'].unique()
    unique_sexos = data['sexo'].unique()

    projections = []

    for cod_ibge in unique_cod_ibge:
        for sexo in unique_sexos:
            data_subset = data[(data['cod_ibge'] == cod_ibge) & (data['sexo'] == sexo)]

            if data_subset.empty or 'ano' not in data_subset.columns:
                continue

            data_subset = data_subset.sort_values(by='ano')
            forecast_values = {}
            for col in columns_to_project:
                series = data_subset[data_subset['ano'] < pd.Timestamp(year, 1, 1)][col]
                model = ExponentialSmoothing(series, trend='add', seasonal=None)
                model_fit = model.fit()
                forecast_value = model_fit.forecast(steps=1).values[0]  # Extrair o valor da projeção

                # Arredonde para número inteiro
                forecast_value = round(forecast_value, 0)

                forecast_values[col] = forecast_value

            projection_dict = {
                'ano': year,
                'cod_ibge': cod_ibge,
                'sexo': sexo,
            }
            projection_dict.update(forecast_values)

            projections.append(projection_dict)

    projections_df = pd.DataFrame(projections)

    return projections_df

columns_to_project = ['qt_0_4', 'qt_5_9', 'qt_10_11', 'qt_12_14', 'qt_15_17', 'qt_18_19', 'qt_20_24', 'qt_25_29', 'qt_30_34',
                  'qt_35_39', 'qt_40_44', 'qt_45_49', 'qt_50_54', 'qt_55_59', 'qt_60_64', 'qt_65_69', 'qt_70_74', 'qt_75_79', 'qt_80_ou_mais']

projections_df = project_cod_ibge_sex(data, columns_to_project, year_to_project)

combined_data = pd.concat([data, projections_df], ignore_index=True)

# Arredonda todas as colunas para números inteiros
combined_data = combined_data.round(0)

# Salva o resultado em um arquivo CSV
combined_data.to_csv('tttttt.csv', index=False)




In [ ]:
combined_data.tail(5)

,cod_ibge,ano,sexo,total,qt_0_4,qt_5_9,qt_10_11,qt_12_14,qt_15_17,qt_18_19,...,qt_35_39,qt_40_44,qt_45_49,qt_50_54,qt_55_59,qt_60_64,qt_65_69,qt_70_74,qt_75_79,qt_80_ou_mais
7665,4207759,2022,Feminino,NaN,102.0,101.0,34.0,52.0,53.0,34.0,...,112.0,122.0,127.0,133.0,141.0,123.0,112.0,92.0,81.0,112.0
7666,4215109,2022,Masculino,NaN,309.0,338.0,131.0,194.0,185.0,116.0,...,503.0,447.0,402.0,442.0,421.0,389.0,308.0,226.0,146.0,159.0
7667,4215109,2022,Feminino,NaN,294.0,306.0,124.0,185.0,185.0,114.0,...,453.0,425.0,382.0,398.0,420.0,381.0,311.0,264.0,190.0,232.0
7668,4218400,2022,Masculino,NaN,207.0,211.0,74.0,113.0,111.0,71.0,...,279.0,250.0,248.0,254.0,262.0,222.0,149.0,116.0,80.0,70.0
7669,4218400,2022,Feminino,NaN,197.0,197.0,78.0,116.0,114.0,70.0,...,289.0,191.0,236.0,277.0,262.0,186.0,161.0,130.0,82.0,116.0


# **CARREGANDO A SEGUNDA BASE DE DADOS PARA PROJETAR 2022**

In [ ]:
df = pd.read_csv('/content/tttttt.csv', delimiter=',')
df['ano'] = pd.to_datetime(df['ano']).dt.strftime('%Y')  # Transforme a coluna 'ano' em um formato de ano YYYY
df['ano'] = df['ano'].astype(int)  # Converta o ano para inteiro
df['ano'] = pd.to_datetime(df['ano'], format='%Y')
df

# Defina o ano para o qual você deseja fazer projeções
year_to_project = 2023

def project_cod_ibge_sex(df, columns_to_project, year):
    unique_cod_ibge = df['cod_ibge'].unique()
    unique_sexos = df['sexo'].unique()

    projections = []

    for cod_ibge in unique_cod_ibge:
        for sexo in unique_sexos:
            data_subset = df[(df['cod_ibge'] == cod_ibge) & (df['sexo'] == sexo)]

            if data_subset.empty or 'ano' not in data_subset.columns:
                continue

            data_subset = data_subset.sort_values(by='ano')
            forecast_values = {}
            for col in columns_to_project:
                series = data_subset[data_subset['ano'] < pd.Timestamp(year, 1, 1)][col]
                model = ExponentialSmoothing(series, trend='add', seasonal=None)
                model_fit = model.fit()
                forecast_value = model_fit.forecast(steps=1).values[0]  # Extrair o valor da projeção

                # Arredonde para número inteiro
                forecast_value = round(forecast_value, 0)

                forecast_values[col] = forecast_value

            projection_dict = {
                'ano': year,
                'cod_ibge': cod_ibge,
                'sexo': sexo,
            }
            projection_dict.update(forecast_values)

            projections.append(projection_dict)

    projections_df = pd.DataFrame(projections)

    return projections_df

columns_to_project = ['qt_0_4', 'qt_5_9', 'qt_10_11', 'qt_12_14', 'qt_15_17', 'qt_18_19', 'qt_20_24', 'qt_25_29', 'qt_30_34',
                  'qt_35_39', 'qt_40_44', 'qt_45_49', 'qt_50_54', 'qt_55_59', 'qt_60_64', 'qt_65_69', 'qt_70_74', 'qt_75_79', 'qt_80_ou_mais']

projections_df = project_cod_ibge_sex(df, columns_to_project, year_to_project)

combined_data = pd.concat([df, projections_df], ignore_index=True)

# Arredonda todas as colunas para números inteiros
combined_data = combined_data.round(0)


# Salva o resultado em um arquivo CSV
combined_data.to_csv('resultado.csv', index=False)


In [ ]:
combined_data

,cod_ibge,ano,sexo,total,qt_0_4,qt_5_9,qt_10_11,qt_12_14,qt_15_17,qt_18_19,...,qt_35_39,qt_40_44,qt_45_49,qt_50_54,qt_55_59,qt_60_64,qt_65_69,qt_70_74,qt_75_79,qt_80_ou_mais
0,4200051,2010-01-01 00:00:00,Masculino,1408.0,86.0,111.0,45.0,69.0,81.0,51.0,...,86.0,114.0,105.0,94.0,93.0,62.0,66.0,31.0,22.0,15.0
1,4200051,2010-01-01 00:00:00,Feminino,1289.0,90.0,83.0,42.0,66.0,66.0,43.0,...,94.0,101.0,79.0,89.0,87.0,73.0,44.0,29.0,17.0,21.0
2,4200101,2010-01-01 00:00:00,Masculino,8776.0,692.0,858.0,398.0,604.0,510.0,309.0,...,588.0,601.0,557.0,471.0,393.0,299.0,217.0,138.0,83.0,63.0
3,4200101,2010-01-01 00:00:00,Feminino,8610.0,689.0,775.0,361.0,555.0,498.0,310.0,...,605.0,579.0,538.0,450.0,364.0,305.0,196.0,138.0,95.0,98.0
4,4200200,2010-01-01 00:00:00,Masculino,4779.0,386.0,342.0,146.0,232.0,262.0,176.0,...,344.0,368.0,325.0,268.0,201.0,162.0,145.0,88.0,73.0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8255,4207759,2023,Feminino,NaN,100.0,101.0,31.0,50.0,51.0,31.0,...,109.0,119.0,123.0,130.0,143.0,125.0,114.0,94.0,83.0,118.0
8256,4215109,2023,Masculino,NaN,309.0,341.0,129.0,192.0,188.0,110.0,...,511.0,460.0,402.0,447.0,429.0,400.0,318.0,235.0,151.0,166.0
8257,4215109,2023,Feminino,NaN,293.0,305.0,123.0,181.0,187.0,109.0,...,458.0,434.0,379.0,395.0,422.0,390.0,319.0,275.0,198.0,239.0
8258,4218400,2023,Masculino,NaN,205.0,212.0,71.0,113.0,110.0,66.0,...,286.0,249.0,247.0,254.0,264.0,231.0,152.0,119.0,83.0,74.0
